In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
file_path = './Final_data.csv'
data = pd.read_csv(file_path,encoding = 'utf-8-sig')

In [ ]:
data.columns

Index(['Type', 'Sentence', 'Quality', 'Price', 'Packaging', 'Service',
       'Sentiment'],
      dtype='object')

In [ ]:
df = []
for i, row in data.iterrows():
  check = True
  for index in range(4):
    if row.iloc[index + 2] != 0:
      df.append([row['Sentence'] , row['Sentiment'], index + 1])
      check = False
  if check:
    df.append([row['Sentence'] , row['Sentiment'], 0])

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# Remove Word have NOT
blabla = ['weren',"weren't",'doesn','hasn',"hasn't",'haven',"haven't","isn't",
          "won't",'wouldn',"wouldn't",
          "aren't", "couldn't",'couldn', 'didn', "didn't",'doesn',"doesn't",'don',"don't",'hadn',"hadn't",
          'isn',"isn't",'mightn',"mightn't",'mustn',"mustn't",'needn',"needn't",'shan',"shan't",'shouldn',
          "shouldn't",'wasn',"wasn't", "didn't",'not', 'nor','aren']
for word in blabla:
  if(word in stop_words):
    stop_words.remove(word)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.stem import LancasterStemmer
wnl = WordNetLemmatizer()
stemmer = LancasterStemmer()

# Word #

word_dict = {"n't" : " not","'re" : " are","'s" : " is", "'ve" : " have"}
def replaceWord(Word):
  w = ''
  run = False
  for word in Word:
    if word == "'":
      run = True
  if run:
    for word in word_dict:
      Word = Word.replace(word,word_dict[word])
  return Word

# Sentences #

def replaceDot(Sen):
  w = ''
  for word in Sen:
    if (ord(word) in range(33,48)) or (ord(word) in range(58,65)) or (ord(word) in range(91,97)):
      if word == "'":
        w += word
      else:
        w += '.'
    else:
      w += word.lower()
  return replaceWord(w)

Links = ['and','but','howerver']
def splitSen(Sen):
  temp = Sen.split('.')
  temp_2 = []
  for sen in temp:
    s = ''
    for word in sen.split():
      if word in Links:
        temp_2.append(s[:-1])
        s = ''
      else:
        s += word + ' '
    if s != '':
      temp_2.append(s[:-1])

  return temp_2

def removeStopWord(Sen):
  s = ''
  for word in Sen.split():
    if word not in stop_words:
      s += stemmer.stem(word) + ' '

  return s[:-1]

# Main Pre-processing Sen #

def ProcessReview(Review):
  Review = replaceDot(Review)
  temp = splitSen(Review)
  for index in range(len(temp)):
    temp[index] = removeStopWord(temp[index])

  return temp

In [ ]:
for index in range(len(df)):
  try:
    df[index][0] =  removeStopWord(df[index][0])
  except:
    print(index)

In [ ]:
cols = ['Key','Amount','Negative','Neutral','Positive','None','Quality','Price','Packaging','Service']
word_code = {}
code = 0
Keys = []

for index in range(len(df)):
  for word in df[index][0].split():
    lb = [1.,0.,0.,0.,0.,0.,0.,0.,0.]
    lb[df[index][1] + 1] += 1
    lb[df[index][2] + 4] += 1
    if word not in word_code:
      word_code[word] = code
      code += 1
      Keys.append([word] + lb)
    else:
      for i in range(8):
        Keys[word_code[word]][i + 1] += lb[i]


In [ ]:
Keys = np.array(Keys)
Keys = pd.DataFrame(Keys,columns = cols)
Keys.iloc[:,1:] = Keys.iloc[:,1:].astype(float)

In [ ]:
Keys.describe()

,Key,Amount,Negative,Neutral,Positive,None,Quality,Price,Packaging,Service
count,2141,2141.0,2141.0,2141.0,2141.0,2141.0,2141.0,2141.0,2141.0,2141.0
unique,2141,53.0,28.0,25.0,35.0,18.0,28.0,18.0,15.0,2.0
top,rec,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
freq,1,1261.0,1145.0,1106.0,1133.0,1349.0,1055.0,1629.0,1678.0,1839.0


In [ ]:
Keys_2 = Keys.copy()
for index in range(Keys.shape[0]):
  temp = Keys_2.iloc[index,2:5].sum()
  if temp != 0:
    Keys_2.iloc[index,2:5] /= temp
  temp = Keys_2.iloc[index,-5:].sum()
  if temp != 0:
    Keys_2.iloc[index,-5:] /= temp

Keys_2.describe()

,Key,Amount,Negative,Neutral,Positive,None,Quality,Price,Packaging,Service
count,2141,2141.0,2141.0,2141.0,2141.0,2141.0,2141.0,2141.0,2141.0,2141.0
unique,2141,53.0,103.0,96.0,104.0,82.0,93.0,78.0,72.0,21.0
top,rec,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
freq,1,1261.0,1145.0,1106.0,1133.0,1349.0,1055.0,1629.0,1678.0,1839.0


In [ ]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler(feature_range=(0, 5))

# Keys_2[cols[2:]]= scaler.fit_transform(Keys_2[cols[2:]])
# Keys_2.describe()

In [ ]:
def sen2vec(Sen,ver = 0):
  temp = pd.DataFrame(columns = cols)
  for word in Sen.split():
    if Keys_2.loc[Keys_2['Key'] == word].empty == False:
      temp = pd.concat([temp,Keys_2.loc[Keys_2['Key'] == word]], axis = 0)

  if temp.empty:
    vec = [0,0,0,0,0,0,0,0]
    return np.array(vec).reshape(1,8)

  result = temp.iloc[:,2:].sum().values.reshape(1,8)
  if ver != 0:
    result[2:] = result[2:] / temp.shape[0]
  return result[0]

# Main #

X = sen2vec(df[0][0])
y = np.array([df[0][1],df[0][2]]).reshape(1,2)
for index in range(len(df)):
  X = np.vstack((X,sen2vec(df[index][0])))
  y = np.vstack((y,np.array([df[index][1],df[index][2]]).reshape(1,2)))

In [ ]:
y

array([[2, 1],
       [2, 1],
       [2, 3],
       ...,
       [2, 2],
       [2, 2],
       [2, 2]])

In [ ]:
# X_sentiment = X[:,:3]
# X_aspect = X[:,3:]
# y_sentiment = y[:,0]
# y_aspect = y[:,1]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state = 42)

In [ ]:
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn import svm
# from sklearn.neighbors import KNeighborsClassifier

# param_grid = {'n_neighbors': [1,3, 5, 7,9,11,13,15], 'weights': ['uniform', 'distance']}

# model = KNeighborsClassifier()

# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
# grid_search.fit(X_train[:,3:], y_train[:,1])

# results = pd.DataFrame(grid_search.cv_results_)

# # Print the results
# print(results[['param_n_neighbors', 'param_weights', 'mean_test_score']])

In [ ]:
from sklearn import svm

model_aspect = svm.SVC(kernel = 'linear',gamma = 0.1, C = 100)
model_aspect.fit(X_train[:,3:],y_train[:,1])

model_sentiment = []
for aspect in range(5):
  temp = []
  for index in range(len(y_train)):
    if y_train[index,1] == aspect:
      temp.append(index)
  x_temp = X_train[temp,:3]
  y_temp = y_train[temp,0]
  model = svm.SVC(kernel = 'linear',gamma = 0.1, C = 100)
  model.fit(x_temp,y_temp)
  model_sentiment.append(model)

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier

# model_aspect = KNeighborsClassifier(n_neighbors = 3)
# model_aspect.fit(X_train[:,3:],y_train[:,1])

# model_sentiment = []
# for aspect in range(5):
#   temp = []
#   for index in range(len(y_train)):
#     if y_train[index,1] == aspect:
#       temp.append(index)
#   x_temp = X_train[temp,:3]
#   y_temp = y_train[temp,0]
#   model = KNeighborsClassifier(n_neighbors = 3)
#   model.fit(x_temp,y_temp)
#   model_sentiment.append(model)

In [ ]:
# from sklearn.linear_model import LogisticRegression

# model_aspect = LogisticRegression()
# model_aspect.fit(X_train[:,3:],y_train[:,1])

# model_sentiment = []
# for aspect in range(5):
#   temp = []
#   for index in range(len(y_train)):
#     if y_train[index,1] == aspect:
#       temp.append(index)
#   x_temp = X_train[temp,:3]
#   y_temp = y_train[temp,0]
#   model = LogisticRegression()
#   model.fit(x_temp,y_temp)
#   model_sentiment.append(model)

In [ ]:
def predict(Vec):
  aspect = model_aspect.predict(Vec[3:].reshape(1,5))
  sentiment = model_sentiment[aspect[0]].predict(Vec[:3].reshape(1,3))

  return [sentiment[0],aspect[0]]

y_pred = []
for item in X_test:
  y_pred.append(predict(item.reshape(1,8)[0]))

y_pred = np.array(y_pred).reshape(-1,2)

In [ ]:
def cal_accuracy(matrix,ver = 0):
  sum = matrix[0][0] + matrix[0][1] + matrix[1][0] + matrix[1][1]
  sum_pos = matrix[1][0] + matrix[1][1]

  accuracy = (matrix[0][0] + matrix[1][1]) / sum
  precision = matrix[1][1] / (matrix[1][1] + matrix[0][1])
  recall = matrix[1][1] / (matrix[1][1] + matrix[1][0])
  f1_score = (2 * precision * recall) / (precision + recall)

  if ver == 0:
    return np.array([accuracy,precision,recall,f1_score]).reshape(1,4) * sum_pos / sum
  return np.array([accuracy,precision,recall,f1_score]).reshape(1,4)

# sentiment accuracty #
result = np.array([0.,0.,0.,0.]).reshape(1,4)
for item in range(3):
  matrix = [[0,0],[0,0]]
  for index in range(len(y_test)):
    a = 1 if y_test[index,0] == item else 0
    b = 1 if y_pred[index,0] == item else 0
    matrix[a][b] += 1
  result = result + cal_accuracy(matrix)
print('sentiment',result[0])

for item in range(5):
  matrix = [[0,0],[0,0]]
  for index in range(len(y_test)):
    a = 1 if y_test[index,1] == item else 0
    b = 1 if y_pred[index,1] == item else 0
    matrix[a][b] += 1
  print(item,cal_accuracy(matrix,1)[0])

sentiment [0.9728832  0.95953209 0.95934959 0.95925035]
0 [0.93495935 0.8490566  0.8490566  0.8490566 ]
1 [0.92682927 0.8902439  0.8902439  0.8902439 ]
2 [0.98373984 0.94444444 0.94444444 0.94444444]
3 [0.95528455 0.73529412 0.92592593 0.81967213]
4 [0.93089431 0.87804878 0.75       0.80898876]


In [ ]:
from sklearn.metrics import classification_report

print('sentiment')
print(classification_report(y_test[:,0],y_pred[:,0], digits=4))
print('aspect')
print(classification_report(y_test[:,1],y_pred[:,1], digits=4))

sentiment
              precision    recall  f1-score   support

           0     0.9518    0.9753    0.9634        81
           1     0.9692    0.9265    0.9474        68
           2     0.9592    0.9691    0.9641        97

    accuracy                         0.9593       246
   macro avg     0.9601    0.9570    0.9583       246
weighted avg     0.9595    0.9593    0.9593       246

aspect
              precision    recall  f1-score   support

           0     0.8491    0.8491    0.8491        53
           1     0.8902    0.8902    0.8902        82
           2     0.9444    0.9444    0.9444        36
           3     0.7353    0.9259    0.8197        27
           4     0.8780    0.7500    0.8090        48

    accuracy                         0.8659       246
   macro avg     0.8594    0.8719    0.8625       246
weighted avg     0.8699    0.8659    0.8657       246



In [ ]:
count = 0
for index in range(len(y_test)):
  if y_test[index][0] == y_pred[index][0] and y_test[index][1] == y_pred[index][1]:
    count += 1
count / len(y_test)

0.8373983739837398

In [ ]:
def analyReview(Review):
  sens = ProcessReview(Review)
  sens_v2 = []
  for sen in sens:
    if sen.isspace() == False and sen != '':
      sens_v2.append(sen)

  lb = []
  for sen in sens_v2:
    lb.append(predict(sen2vec(sen)))

  for index in range(len(lb)):
    print(sens_v2[index],lb[index])

text = "The price was great, but the taste is not great. The expiration date is 2023. It’s flat."
# text = "Nothing fancy here, just exactly what you'd expect - 16.9oz bottles of Coca Cola. Saved me a trip to the store. Definitely worth."

analyReview(text)

pric gre [2, 2]
tast not gre [2, 1]
expir dat 2023 [0, 3]
it’s flat [0, 1]
